# prep

In [28]:
import telebot
from telebot import types
import torch
from PIL import Image
import numpy as np
from torchvision import transforms
from io import BytesIO

In [29]:
TOKEN = '6031889345:AAFYZ-Sih2PJBbWf8_QUnBnt1Qd3LzpK_O4'
bot = telebot.TeleBot(TOKEN)

In [30]:
arr = {
    0: 'Аневризматическая дилатация брюшной аорты',
    1: 'Аневризматическая дилатация аорты',
    2: 'Аневризма аорты',
    3: 'Рак',
    4: 'Формирование надпочечника',
    5: 'Злокачественное развитие почек',
    6: 'Злокачественное формирование печени',
    7: 'Мочекаменная болезнь',
    8: 'Компрессионный перелом позвонков'
}

# model integration

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
best_model = torch.load("modd.pth")
best_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# data prep

In [23]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# test, BOT

In [24]:
def predict_image(image, threshold=0.5):
    # Преобразование изображения и подготовка к передаче в модель
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = best_model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)  # Преобразование в вероятности
        max_prob, predicted_index = torch.max(probabilities, 1)  # Нахождение максимальной вероятности и индекса
    
    # Проверка, превышает ли максимальная вероятность заданный порог
    if max_prob.item() < threshold:
        return None  # Возвращение None или другого индикатора недостаточной уверенности
    else:
        return predicted_index.item()  

In [25]:
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, "Привет! Отправь мне изображение, и я попытаюсь его классифицировать")

In [26]:
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    try:
        file_info = bot.get_file(message.photo[-1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        image_bytes = BytesIO(downloaded_file)
        image = Image.open(image_bytes).convert("RGB")

        prediction = predict_image(image)
        if prediction is None:
            response_message = "Ошибка распознавания: модель не уверена в предсказании"
        else:
            response_message = f"Модель считает это - {arr[prediction]}"
        
        bot.reply_to(message, response_message)
    except Exception as e:
        bot.reply_to(message, f'Произошла ошибка при обработке вашего изображения: {e}')

In [27]:
if __name__ == '__main__':
    bot.polling(none_stop=True)

ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)